# All Domains

In [1]:
import pickle
from collections import Counter
import re
import ipy_table
import nltk
import os
import numpy as np
import spacy
import random
from spacy.pipeline import Sentencizer
import json
#from matplotlib import pyplot
import pandas as pd
import xmltodict
from dicttoxml import dicttoxml
from xml.dom.minidom import parseString

import warnings
warnings.filterwarnings('ignore')

2021-12-23 20:44:28.186403: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-23 20:44:28.186442: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
%run ../funcoes.py

Loading functions...


In [3]:
# spacy model
nlp = spacy.load("pt_core_news_lg")

sentencizer = nlp.create_pipe("sentencizer")
punct = [x for x in sentencizer.punct_chars]
punct.append('...')
punct.append('..')
config = {"punct_chars": punct}

#sentencizer = Sentencizer(punct_chars=punct)
nlp.add_pipe("sentencizer", first=True, config=config)

print(nlp.pipe_names)

['sentencizer', 'tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [4]:
# vargas datasets
with open('../data/aspects_camera.dat', "rb") as f: implicitCamera = pickle.load(f)
with open('../data/aspects_livro.dat', "rb") as f: implicitLivro = pickle.load(f)
with open('../data/aspects_smartphone.dat', "rb") as f: implicitSmartphone = pickle.load(f)
with open('../data/aspects_hotel.dat', "rb") as f: implicitHotel = pickle.load(f)

In [5]:
vargasDatasets= {
    'camera'     : implicitCamera,
    'livro'      : implicitLivro,
    'smartphone' : implicitSmartphone,
    'hotel'      : implicitHotel
}
datasets = [ 'camera', 'livro', 'smartphone', 'hotel' ]

In [6]:
f = open("../07_Keras/data/aspectsTest.xml","r")
xml = f.read()
aspectsTest = xmltodict.parse(xml, process_namespaces=True)

In [7]:
tag_scheme = [
'B-asp',
'I-asp'
]

In [8]:
training = {'sentences': [], 'tags' : []}
validation = {'sentences': [], 'tags' :[]}

## Cameras

In [9]:
dfCameraTest = pd.read_csv('../07_Keras/data/domain_camera_IOB_test.csv', sep='\t', quoting=3, keep_default_na=False)
dfCameraTrain = pd.read_csv('../07_Keras/data/domain_camera_IOB_train.csv', sep='\t', quoting=3, keep_default_na=False)

contSents = 0
for count, row in dfCameraTrain.iterrows():
    if (row[0] != "" and count > 0):
        contSents = contSents + 1
contSents = contSents + 1
contTrain = round(contSents * 0.9)

sent = []
tags = []
contSents = 0
for count, row in dfCameraTrain.iterrows():
    if (row[0] != "" and count > 0):
        if contSents < contTrain:
            training['sentences'].append(sent)
            training['tags'].append(tags)
        else:
            validation['sentences'].append(sent)
            validation['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
validation['sentences'].append(sent)
validation['tags'].append(tags)
sent = []
tags = []

cameraTest = {'sentences': [], 'tags' : []}
contSents = 0
for count, row in dfCameraTest.iterrows():
    if (row[0] != "" and count > 0):
        cameraTest['sentences'].append(sent)
        cameraTest['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
cameraTest['sentences'].append(sent)
cameraTest['tags'].append(tags)

## Livros

In [10]:
dfLivroTest = pd.read_csv('../07_Keras/data/domain_livro_IOB_test.csv', sep='\t', quoting=3, keep_default_na=False)
dfLivroTrain = pd.read_csv('../07_Keras/data/domain_livro_IOB_train.csv', sep='\t', quoting=3, keep_default_na=False)

contSents = 0
for count, row in dfLivroTrain.iterrows():
    if (row[0] != "" and count > 0):
        contSents = contSents + 1
contSents = contSents + 1

contTrain = round(contSents * 0.9)

sent = []
tags = []
contSents = 0
for count, row in dfLivroTrain.iterrows():
    if (row[0] != "" and count > 0):
        if contSents < contTrain:
            training['sentences'].append(sent)
            training['tags'].append(tags)
        else:
            validation['sentences'].append(sent)
            validation['tags'].append(tags)

        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
        
validation['sentences'].append(sent)
validation['tags'].append(tags)

livroTest = {'sentences': [], 'tags' : []}
sent = []
tags = []
contSents = 0
for count, row in dfLivroTest.iterrows():
    if (row[0] != "" and count > 0):
        livroTest['sentences'].append(sent)
        livroTest['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
livroTest['sentences'].append(sent)
livroTest['tags'].append(tags)

## Smartphones

In [11]:
dfSmartphoneTest = pd.read_csv('../07_Keras/data/domain_smartphone_IOB_test.csv', sep='\t', quoting=3, keep_default_na=False)
dfSmartphoneTrain = pd.read_csv('../07_Keras/data/domain_smartphone_IOB_train.csv', sep='\t', quoting=3, keep_default_na=False)

contSents = 0
for count, row in dfSmartphoneTrain.iterrows():
    if (row[0] != "" and count > 0):
        contSents = contSents + 1
contSents = contSents + 1

contTrain = round(contSents * 0.9)

sent = []
tags = []
contSents = 0
for count, row in dfSmartphoneTrain.iterrows():
    if (row[0] != "" and count > 0):
        if contSents < contTrain:
            training['sentences'].append(sent)
            training['tags'].append(tags)
        else:
            validation['sentences'].append(sent)
            validation['tags'].append(tags)

        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
        
validation['sentences'].append(sent)
validation['tags'].append(tags)

smartphoneTest = {'sentences': [], 'tags' : []}
sent = []
tags = []
contSents = 0
for count, row in dfSmartphoneTest.iterrows():
    if (row[0] != "" and count > 0):
        smartphoneTest['sentences'].append(sent)
        smartphoneTest['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
smartphoneTest['sentences'].append(sent)
smartphoneTest['tags'].append(tags)

### Hotel

In [12]:
dfHotelTest = pd.read_csv('../07_Keras/data/domain_hotel_IOB_test.csv', sep='\t', quoting=3, keep_default_na=False)
dfHotelTrain = pd.read_csv('../07_Keras/data/domain_hotel_IOB_train.csv', sep='\t', quoting=3, keep_default_na=False)

contSents = 0
for count, row in dfHotelTrain.iterrows():
    if (row[0] != "" and count > 0):
        contSents = contSents + 1
contSents = contSents + 1

contTrain = round(contSents * 0.9)

sent = []
tags = []
contSents = 0
for count, row in dfHotelTrain.iterrows():
    if (row[0] != "" and count > 0):
        if contSents < contTrain:
            training['sentences'].append(sent)
            training['tags'].append(tags)
        else:
            validation['sentences'].append(sent)
            validation['tags'].append(tags)

        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
        
validation['sentences'].append(sent)
validation['tags'].append(tags)

hotelTest = {'sentences': [], 'tags' : []}
sent = []
tags = []
contSents = 0
for count, row in dfHotelTest.iterrows():
    if (row[0] != "" and count > 0):
        hotelTest['sentences'].append(sent)
        hotelTest['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
hotelTest['sentences'].append(sent)
hotelTest['tags'].append(tags)

In [13]:
print(len(training['sentences']))
print(len(validation['sentences']))

print(len(cameraTest['sentences']))
print(len(livroTest['sentences']))
print(len(smartphoneTest['sentences']))

1230
136
81
88
106


## Bertimbau

In [14]:
transformer = 'neuralmind/bert-base-portuguese-cased'

In [15]:
# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
    'epochs' : 40,
    'warmup_steps' : 500,                                                   
    'train_batch_size': 13,
    'learning_rate': 0.0001
}

In [16]:
from NERDA.models import NERDA
model = NERDA(
    dataset_training = training,
    dataset_validation = validation,
    tag_scheme = tag_scheme, 
    tag_outside = 'O',
    transformer = transformer,
    dropout = dropout,
    hyperparameters = training_hyperparameters
)

Device automatically set to: cpu


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
%%time
model.train()


 Epoch 1 / 40


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:18<00:00,  1.06s/it]


Train Loss = 0.48635165487465104 Valid Loss = 0.287363663315773

 Epoch 2 / 40


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:17<00:00,  1.01s/it]


Train Loss = 0.23877426166283458 Valid Loss = 0.1659946286064737

 Epoch 3 / 40


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:16<00:00,  1.01it/s]


Train Loss = 0.0004493965863060255 Valid Loss = 0.3370998317704481

 Epoch 40 / 40


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:16<00:00,  1.00it/s]

Train Loss = 0.0002849374727750393 Valid Loss = 0.33734343025614233
CPU times: user 1d 21h 50min 45s, sys: 30min 39s, total: 1d 22h 21min 24s
Wall time: 7h 25min 50s


'Model trained successfully'

In [18]:
def result2aspecList(y_pred, X_test):
    aspectsList = []
    aspect = []
    stopwords = nltk.corpus.stopwords.words('portuguese')
    for sent, sent_pred in zip(X_test, y_pred):
        #print(len(sent),len(sent_pred))
        for token, tag in zip(sent, sent_pred):
            if tag != 'O':
                #print(tag, token)
                #print(tag, token['word.lower()'])
                if tag == 'B-asp' and aspect:
                    auxAspect = " ".join(aspect).lower()
                    if (auxAspect not in stopwords and len(auxAspect) > 1):
                        aspectsList.append(auxAspect)
                    aspect = []
                if token not in [',',':']:
                    aspect.append(token.lower())

    aspectsList = list(set(aspectsList))
    aspectsList.sort()    
    return aspectsList

### Cameras

In [19]:
# carregando tipos
domain = "camera"
tiposAll = pd.read_csv('../data/tipologia/categorias_iacs.csv', sep=',', quoting=0, keep_default_na=False)
tipos = tiposAll.loc[tiposAll['domain'] == domain]

In [20]:
model.evaluate_performance(cameraTest)

,Level,F1-Score,Precision,Recall
0,B-asp,0.397959,0.364486,0.438202
1,I-asp,0.311688,0.342857,0.285714
0,AVG_MICRO,0.373626,NaN,NaN
0,AVG_MICRO,0.354824,NaN,NaN


In [21]:
y_pred = model.predict(cameraTest['sentences'])

In [22]:
aspectsCamera = result2aspecList(y_pred, cameraTest['sentences'])
resCamera = calculaMetricas(
    aspectsCamera, 
    aspectsTest['datasets']['camera'], 
    tipos,
    lemma=False, 
    output=False)
resCamera

{'totalAspects': 104,
 'pred': 60,
 'tp': 56,
 'fp': 35,
 'fn': 48,
 'precisao': 0.6153846153846154,
 'precisaoLaplace': 0.6129032258064516,
 'revocacao': 0.5384615384615384,
 'medidaf': 0.5743589743589744,
 'totalExplicitos': 91,
 'explicitosOk': 24,
 'pExplicitos': 0.26373626373626374,
 'explicitsList': ['bateria',
  'cartão de memória',
  'flash',
  'menu',
  'memória',
  'zoom',
  'lente',
  'acessórios',
  'preço',
  'produto',
  'investimento',
  'funções',
  'praticidade',
  'botão',
  'câmera',
  'fotografia',
  'máquina',
  'camera',
  'fotos',
  'resolução',
  'visor',
  'manuseio',
  'recursos',
  'qualidade de imagem'],
 'totalImplicitos': 13,
 'implicitosOk': 1,
 'pImplicitos': 0.07692307692307693,
 'implicitsList': ['compacta'],
 'tipos': {'Qualification': {'Adjective': 4}}}

In [23]:
xml = dicttoxml(resCamera, attr_type=False, custom_root='resultados')
with open('../resultados/bert_bertimbau_alldomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())    

## Livros

In [24]:
# carregando tipos
domain = "livro"
tiposAll = pd.read_csv('../data/tipologia/categorias_iacs.csv', sep=',', quoting=0, keep_default_na=False)
tipos = tiposAll.loc[tiposAll['domain'] == domain]

In [25]:
model.evaluate_performance(livroTest)

,Level,F1-Score,Precision,Recall
0,B-asp,0.742138,0.797297,0.694118
1,I-asp,0.153846,0.500000,0.090909
0,AVG_MICRO,0.697674,NaN,NaN
0,AVG_MICRO,0.447992,NaN,NaN


In [26]:
y_pred = model.predict(livroTest['sentences'])

In [27]:
aspectsLivro = result2aspecList(y_pred, livroTest['sentences'])
resLivro = calculaMetricas(
    aspectsLivro, 
    aspectsTest['datasets'][domain], 
    tipos,
    output=False)
resLivro

{'totalAspects': 87,
 'pred': 29,
 'tp': 55,
 'fp': 9,
 'fn': 32,
 'precisao': 0.859375,
 'precisaoLaplace': 0.8484848484848485,
 'revocacao': 0.632183908045977,
 'medidaf': 0.7284768211920529,
 'totalExplicitos': 82,
 'explicitosOk': 19,
 'pExplicitos': 0.23170731707317074,
 'explicitsList': ['história',
  'escritoras',
  'trama',
  'narrativa',
  'leitura',
  'historia',
  'personagens',
  'livro',
  'fim',
  'começo',
  'palavra',
  'situaçoês',
  'início',
  'estilo',
  'personagem',
  'final',
  'obra',
  'narrador',
  'romance'],
 'totalImplicitos': 5,
 'implicitosOk': 1,
 'pImplicitos': 0.2,
 'implicitsList': ['escreve'],
 'tipos': {'Event': {'Verb': 1}}}

In [28]:
xml = dicttoxml(resLivro, attr_type=False, custom_root='resultados')
with open('../resultados/bert_bertimbau_alldomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())

## Smartphones

In [29]:
# carregando tipos
domain = "smartphone"
tiposAll = pd.read_csv('../data/tipologia/categorias_iacs.csv', sep=',', quoting=0, keep_default_na=False)
tipos = tiposAll.loc[tiposAll['domain'] == domain]

In [30]:
model.evaluate_performance(smartphoneTest)

,Level,F1-Score,Precision,Recall
0,B-asp,0.441261,0.445087,0.4375
1,I-asp,0.359551,0.410256,0.3200
0,AVG_MICRO,0.424658,NaN,NaN
0,AVG_MICRO,0.400406,NaN,NaN


In [31]:
y_pred = model.predict(smartphoneTest['sentences'])

In [32]:
aspectsSmartphone = result2aspecList(y_pred, smartphoneTest['sentences'])
resSmartphone = calculaMetricas(
    aspectsSmartphone, 
    aspectsTest['datasets'][domain], 
    tipos,
    lemma=False, output=False)
resSmartphone

{'totalAspects': 158,
 'pred': 83,
 'tp': 84,
 'fp': 42,
 'fn': 74,
 'precisao': 0.6666666666666666,
 'precisaoLaplace': 0.6640625,
 'revocacao': 0.5316455696202531,
 'medidaf': 0.5915492957746478,
 'totalExplicitos': 134,
 'explicitosOk': 38,
 'pExplicitos': 0.2835820895522388,
 'explicitsList': ['smartphone',
  'celular',
  'durabilidade',
  'designer',
  'preço',
  'produto',
  'funções',
  'e-mail',
  'marca',
  'recursos',
  'bateria',
  'menu',
  'touch',
  'camera digital',
  'qualidade de som',
  'internet',
  'wifi',
  'funcionalidades',
  'volume de áudio',
  'aparelho',
  'design',
  'fabricante',
  'qualidade das fotos',
  'memoria',
  'wi-fi',
  'gps',
  'recurso',
  'tamanho',
  'fotos',
  'visor',
  'nokia',
  'custo-benefício',
  'memoria interna',
  'modelo',
  'display',
  'câmera',
  'camera',
  'sd de memória'],
 'totalImplicitos': 24,
 'implicitosOk': 3,
 'pImplicitos': 0.125,
 'implicitsList': ['lento', 'trava', 'prático'],
 'tipos': {'Qualification': {'Adjective'

In [33]:
xml = dicttoxml(resSmartphone, attr_type=False, custom_root='resultados')
with open('../resultados/bert_bertimbau_alldomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())

## Hotel

In [34]:
# carregando tipos
domain = "hotel"
tiposAll = pd.read_csv('../data/tipologia/categorias_iacs.csv', sep=',', quoting=0, keep_default_na=False)
tipos = tiposAll.loc[tiposAll['domain'] == domain]

In [35]:
model.evaluate_performance(hotelTest)

,Level,F1-Score,Precision,Recall
0,B-asp,0.780612,0.794805,0.766917
1,I-asp,0.477612,0.744186,0.351648
0,AVG_MICRO,0.736383,NaN,NaN
0,AVG_MICRO,0.629112,NaN,NaN


In [36]:
y_pred = model.predict(hotelTest['sentences'])

In [37]:
aspectsHotel = result2aspecList(y_pred, hotelTest['sentences'])
resHotel = calculaMetricas(
    aspectsHotel, 
    aspectsTest['datasets'][domain], 
    tipos,
    lemma=False, output=False)
resHotel

{'totalAspects': 518,
 'pred': 127,
 'tp': 374,
 'fp': 43,
 'fn': 144,
 'precisao': 0.8968824940047961,
 'precisaoLaplace': 0.8949880668257757,
 'revocacao': 0.722007722007722,
 'medidaf': 0.8,
 'totalExplicitos': 292,
 'explicitosOk': 40,
 'pExplicitos': 0.136986301369863,
 'explicitsList': ['lojas',
  'gerência',
  'funcionário',
  'piscina',
  'shopping',
  'localização',
  'preço',
  'chuveiro',
  'garagem',
  'tv',
  'apartamento',
  'portaria',
  'telefone',
  'quarto',
  'limpeza',
  'pia',
  'internet',
  'instalações',
  'estabelecimento',
  'hotel',
  'atendimento',
  'padrão',
  'conforto',
  'estacionamento',
  'cozinha',
  'rua',
  'corredor',
  'rodoviária',
  'elevador',
  'serviço de quarto',
  'cidade',
  'aparência',
  'recepção',
  'serviço',
  'frigobar',
  'funcionários',
  'aeroporto',
  'lavanderia',
  'carpete',
  'porteiro'],
 'totalImplicitos': 226,
 'implicitosOk': 56,
 'pImplicitos': 0.24778761061946902,
 'implicitsList': ['manchada',
  'localizado no centro

In [38]:
xml = dicttoxml(resHotel, attr_type=False, custom_root='resultados')
with open('../resultados/bert_bertimbau_alldomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())